In [1]:
import pyciemss

In [2]:
### SETUP ###

model_1_path = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/raw_models/SEIRHD_NPI_Type1.json"
start_time = 0.0
end_time = 100.
logging_step_size = 10.0
num_samples = 3

In [3]:
### SAMPLE INTERFACE ###

result = pyciemss.sample(model_1_path, end_time, logging_step_size, num_samples, start_time=start_time)
result["unprocessed_result"]

ERROR:root:
                ###############################

                There was an exception in pyciemss

                Error occured in function: sample

                Function docs : 
    Load a model from a file, compile it into a probabilistic program, and sample from it.

    Args:
        model_path_or_json: Union[str, Dict]
            - A path to a AMR model file or JSON containing a model in AMR form.
        end_time: float
            - The end time of the sampled simulation.
        logging_step_size: float
            - The step size to use for logging the trajectory.
        num_samples: int
            - The number of samples to draw from the model.
        solver_method: str
            - The method to use for solving the ODE. See torchdiffeq's `odeint` method for more details.
            - If performance is incredibly slow, we suggest using `euler` to debug.
              If using `euler` results in faster simulation, the issue is likely that the model is s

NotImplementedError: 

In [43]:
result['data'].head()

,timepoint_id,sample_id,beta_c_param,kappa_param,gamma_param,hosp_param,death_hosp_param,D_state,E_state,H_state,I_state,R_state,S_state
0,0,0,0.359898,0.564617,0.149691,0.07073,0.028889,0.068015,148.878113,3.054959,113.107819,73.355171,19339664.0
1,1,0,0.359898,0.564617,0.149691,0.07073,0.028889,0.574769,893.032288,18.941544,678.870483,529.574768,19337882.0
2,2,0,0.359898,0.564617,0.149691,0.07073,0.028889,3.630535,5355.246582,113.715500,4072.014648,3267.171631,19327182.0
3,3,0,0.359898,0.564617,0.149691,0.07073,0.028889,21.950045,31998.847656,681.128418,24368.181641,19669.208984,19263260.0
4,4,0,0.359898,0.564617,0.149691,0.07073,0.028889,131.163406,187168.984375,4041.593506,143826.796875,117159.468750,18887674.0


In [44]:
### ENSEMBLE SETUP ###
model_2_path = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/raw_models/SEIRHD_NPI_Type2.json"

In [45]:
result_2 = pyciemss.sample(model_2_path, end_time, logging_step_size, num_samples, start_time=start_time)
result_2['data'].head()

,timepoint_id,sample_id,beta_c_param,beta_nc_param,kappa_param,gamma_param,hosp_param,death_hosp_param,D_state,E_state,H_state,I_state,R_state,S_state
0,0,0,0.670123,0.24992,0.241247,0.256445,0.157044,0.05834,0.271666,23.428442,3.924912,23.333035,50.447315,19339898.0
1,1,0,0.670123,0.24992,0.241247,0.256445,0.157044,0.05834,0.772988,21.617098,4.411857,21.717606,107.277359,19339844.0
2,2,0,0.670123,0.24992,0.241247,0.256445,0.157044,0.05834,1.276975,20.030890,4.194791,20.125315,160.617111,19339804.0
3,3,0,0.670123,0.24992,0.241247,0.256445,0.157044,0.05834,1.749375,18.561569,3.901538,18.649137,210.131348,19339748.0
4,4,0,0.670123,0.24992,0.241247,0.256445,0.157044,0.05834,2.187848,17.200031,3.617305,17.281147,256.025665,19339704.0


In [47]:
### ENSEMBLE SAMPLE INTERFACE ###

model_paths = [model_1_path, model_2_path]
solution_mappings = [lambda x : x, lambda x : x] # Conveniently, these two models operate on exactly the same state space, with the same names.

ensemble_result = pyciemss.ensemble_sample(model_paths, solution_mappings, end_time, logging_step_size, num_samples, start_time=start_time)
ensemble_result['data'].head()

,timepoint_id,sample_id,model_0/beta_c_param,model_0/kappa_param,model_0/gamma_param,model_0/hosp_param,model_0/death_hosp_param,model_1/beta_c_param,model_1/beta_nc_param,model_1/kappa_param,...,model_0/H_state,model_0/I_state,model_0/R_state,model_0/S_state,model_1/D_state,model_1/E_state,model_1/H_state,model_1/I_state,model_1/R_state,model_1/S_state
0,0,0,0.234028,0.143929,0.285486,0.183518,0.075701,0.330519,0.565437,0.429667,...,3.762744,14.006333,44.255589,19339924.0,0.032969,34.200268,1.644323,20.677273,72.392578,19339870.0
1,1,0,0.234028,0.143929,0.285486,0.183518,0.075701,0.330519,0.565437,0.429667,...,2.485743,6.566550,73.194809,19339904.0,0.097246,37.786564,2.094492,22.866238,159.587280,19339774.0
2,2,0,0.234028,0.143929,0.285486,0.183518,0.075701,0.330519,0.565437,0.429667,...,1.251937,3.034535,87.217476,19339898.0,0.172334,41.763142,2.352500,25.272713,256.201996,19339672.0
3,3,0,0.234028,0.143929,0.285486,0.183518,0.075701,0.330519,0.565437,0.429667,...,0.592406,1.401877,93.778893,19339898.0,0.255871,46.157814,2.605152,27.932184,363.016052,19339556.0
4,4,0,0.234028,0.143929,0.285486,0.183518,0.075701,0.330519,0.565437,0.429667,...,0.275575,0.647627,96.821304,19339898.0,0.348272,51.014397,2.879967,30.871220,481.073914,19339430.0
